In [ ]:
import os 
import json
import requests
import tweepy
import numpy as np
import pandas as pd
from keras.models import load_model
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
consumer_key = '6gokUJ7gZ3ixFNkIRUjxL7Xwf'
consumer_secret = 'Us8FgUedJ610MDlH4ZK0wga1AtY4wNALQdHf50g3pj7Lm3IWAG'
access_token ='542866005-43KxBY08C7knS8VYNAycISB3AyqYmONzt2IhWk3m'
access_token_secret = 'YZ8oJWoyR3SgbIfo1I6PJV5wFbMJSVjIukCwiNPryjXzM'

In [ ]:
jsonPath = r'C:\Users\jame9353\Documents\GitHub\MappingTwitterSentiment\TwitterJSON'
rdfPath = r'C:\Users\jame9353\Documents\GitHub\MappingTwitterSentiment\TwitterRDF'
textPath = r'C:\Users\jame9353\Documents\GitHub\MappingTwitterSentiment\TwitterText'

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [ ]:
api = tweepy.API(auth)

In [ ]:
model = load_model('Twitter_SA_Model.h5')

In [ ]:
def netowlCurl(inFile, outPath, outExtension):
    headers = {
    'accept': 'application/rdf+xml',
    'Authorization': 'netowl ff5e6185-5d63-459b-9765-4ebb905affc8',
    }
    
    
    headers['Content-Type'] = 'text/plain'
        
    params = (
        ('language', 'english'),
    )
    
    data = open(inFile, 'rb').read()
    response = requests.post('https://api.netowl.com/api/v2/_process', headers=headers, params=params, data=data, verify=False)
    r = response.text
    outPath = outPath
    fileName = os.path.split(d)[1]
    if os.path.exists(outPath) == False:
        os.mkdir(outPath, mode=0o777,)
    outFile = os.path.join(outPath, fileName + outExtension)
    open(outFile, "w", encoding="utf-8").write(r)

In [ ]:
with open('dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

In [ ]:
tokenizer = Tokenizer(num_words=3000)

In [ ]:
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
        else:
            print("'%s' not in training corpus; ignoring." %(word))
    return wordIndices

In [ ]:
def Sentiment(tweet_text):
    labels = ['positive', 'negative']
    testArr = convert_text_to_index_array(tweet_text)
    twt = tokenizer.sequences_to_matrix([testArr], mode='binary')
    twt = pad_sequences(twt, maxlen=86, dtype='int32', padding='post', truncating='post', value=0)
    sentiment = model.predict(twt)
    accuracy = sentiment[0][np.argmax(sentiment)] * 100
    tweetSent = labels[np.argmax(sentiment)]
    return tweetSent, accuracy

In [ ]:
htList = []
mentionList = []
users = []
coords = []
replies = []
entities = []
cColumns = ['Tweet ID', 'Coordinates']
rColumns = ['Tweet ID', 'Screen Name', 'Replied To', 'Screen Name']
uColumns = ['User', 'Location', 'Description', 'Tweets', 'Date Created', "Followers", 'Friends']
afrinTweets = []
for tweet in tweepy.Cursor(api.search,q="#Afrin",count=100,lang="en").items():
    iRow = []
    tweetID = tweet.id
    date = tweet.created_at
    text = tweet.text
    user = tweet.user.screen_name
    rt = tweet.retweet_count
    fav = tweet.favorite_count
    location = tweet.user.location
    hashtags = tweet.entities.get('hashtags')
    hashtag = ""
    for n in hashtags:
        htl = []
        hashtag += str(n['text']) + ", "
        htl.append(user)
        htl.append(str(n['text']))
        htList.append(htl)
        
    mentions = tweet.entities.get('user_mentions')
    mention = ""
    for n in mentions:
        mention += str(n['screen_name']) + ", "
        ml = []
        ml.append(user)
        ml.append(str(n['screen_name']))
        mentionList.append(ml)
    tweetSent = ""
    
    iRow.append(tweetID)
    iRow.append(date)
    iRow.append(text)
    iRow.append(user)
    iRow.append(rt)
    iRow.append(fav)
    iRow.append(hashtag)
    iRow.append(mention)
    iRow.append(tweet.retweeted)
    iRow.append(tweetSent)
    
    afrinTweets.append(iRow)

    if user not in users:
        uRow = []
        uRow.append(user)
        uRow.append(location)
        uRow.append(tweet.user.description)
        uRow.append(tweet.user.statuses_count)
        uRow.append(tweet.user.created_at)
        uRow.append(tweet.user.followers_count)
        uRow.append(tweet.user.friends_count)
        users.append(uRow)
        
    if tweet.geo != None:
        cRow = []
        coord = tweet.coordinates.get('coordinates')
        cRow.append(tweetID)
        cRow.append(coord)
        coords.append(cRow)
        
    if tweet.in_reply_to_user_id != None:
        rRow = []
        replyUID = tweet.in_reply_to_user_id
        replySC = tweet.in_reply_to_screen_name
        rRow.append(tweetID)
        rRow.append(user)
        rRow.append(replyUID)
        rRow.append(replySC)
        replies.append(rRow)
        
    tj = tweet._json
    File = str(tweetID) + ".txt"
    tweetFile = os.path.join(textPath, File)
    with open(tweetFile, "w", encoding="utf-8") as note:
        note.write(text)
        note.close()
        
    json = str(tweetID) + ".json"
    outFile = os.path.join(jsonPath, json)
    with open(outFile, "w", encoding="utf-8") as out:
        json.dump(tj, out)

In [ ]:
columns = ['Tweet Id', 'Date', 'Text', 'User', 'Retweets', 'Favorites', 'Hashtags', 'Mentions', 'Retweeted', 'Sentiment']
df = pd.DataFrame(afrinTweets,columns=columns)
usersDF = pd.DataFrame(users, columns=uColumns)
repliesDF = pd.DataFrame(replies, columns=rColumns)
coordsDF = pd.DataFrame(coords, columns=cColumns)

In [ ]:
sentimentList = []
for row in df.iterrows():
    sl = []
    tweetText = row[1]['Text']
    tweetID =row[1]['Tweet Id']
    user = row[1]['User']
    sent = Sentiment(tweetText)
    sl.append(tweetID)
    sl.append(user)
    sl.append(sent[0])
    sl.append(sent[1])
    sentimentList.append(sl)
    
columns = ['Tweet Id', 'Handle', 'Sentiment', 'Confidence']
sentDF = pd.DataFrame(sentimentList,columns=columns)
mask = sentDF.Confidence < 60
column_name = 'Sentiment'
sentDF.loc[mask, column_name] = "neutral"

In [ ]:
usersDF['Location'].replace('', np.nan, inplace=True)
userLocs = usersDF[['User', 'Location']].dropna()

In [ ]:
cleanDF = df[['Tweet Id', 'Date', 'Text', 'User', 'Retweets', 'Favorites', 'Hashtags', 'Mentions', 'Retweeted']]
tweetSent = pd.merge(cleanDF, sentDF, left_on='Tweet Id', right_on='Tweet Id')
tweetDF = tweetSent[['Tweet Id', 'Date', 'Text', 'User', 'Retweets', 'Favorites', 'Hashtags', 'Mentions', 'Retweeted', 'Sentiment', 'Confidence']]

In [ ]:
#Walks through the docsPath, identifying files, and appends them to a list.
docs = []
for root, dirs, files in os.walk(textPath):
    for f in files:
        filePath = os.path.join(root, f)
        docs.append(filePath)
        
#Iterates though the docs list created previously and 
# runs the function for each of the documents found. 
#Passes the function a document derived from the list,
# and two variables created in a previous step. 

for d in docs:
    netowlCurl(d, rdfPath, ".rdf")

In [ ]:
from rdflib import Graph
from gastrodon import LocalEndpoint, one, QName

#Creates a Graph Object that will store all the result of a parse operation 
# in the next step. 
g = Graph()

#Walks through output path from the netowlCurl function and parses all RDF/XML Documents
for root, dir, files in os.walk(rdfPath):
    for file in files:
        if file.endswith('.rdf'):
            filePath = os.path.join(root, file)
            print("Parsing " + file + "...")
            try:
                g.parse(filePath, format='xml')
            except Exception as ex:
                print(ex)
                
#Create Local SPARQL Endpoint on graph created in previous step
e = LocalEndpoint(g)

In [ ]:
len(g)

In [ ]:
import matplotlib.pyplot as plt
#Uses the SPARQL endpoint to query all of the relationship types and returns the top 10
properties=e.select("""
   SELECT ?p ?type(COUNT(*) AS ?cnt) {
      ?s ?p ?o .
      ?s rdf:type ?type . 
   } GROUP BY ?type ORDER BY DESC(?cnt)
""")
properties.head(10)
    
#Writes the full relationship types data frame to a CSV document
file_name = 'predicates.csv'
properties.to_csv(file_name, sep=',', encoding='utf-8')

In [ ]:
#Graphs the top 10 relationship types
with plt.xkcd():
    properties.head(10)["cnt"].plot.pie(figsize=(6,6)).set_ylabel('')

In [ ]:
#Uses the SPARQL endpoint to query all of the relationship types and returns the top 10
triplesSel=e.select("""
   SELECT ?s ?p ?o ?sLabel ?sType ?oLabel ?oType
   WHERE{
      ?s ?p ?o.
      OPTIONAL { ?s rdfs:label ?sLabel }.
      OPTIONAL { ?s rdf:type ?sType } .
      OPTIONAL { ?o rdfs:label ?oLabel} .
      OPTIONAL { ?o rdf:type ?oType} .
    }
""")
triplesSel.to_csv(tripleList, sep=',', encoding='utf-8')
triples = pd.read_csv(tripleList)
triples.head()

In [ ]:
cityQ = triples['sType'] == "netowl:Entity.Place.City"
label = triples['p'] == "rdfs:label"
cities = triples[cityQ & label]

In [ ]:
cities

In [ ]:
from arcgis.gis import GIS
import json
gis = GIS("https://wdcdefense.esri.com/portal", 'james_jones')

In [ ]:
cities_csv = cities.to_csv('Cities.csv')

In [ ]:
csv_item = gis.content.add({'Type':'CSV',
                           'Title':"Mentioned Cities"},
                           cities_csv)
display(csv_item)

In [ ]:
csv_lyr = csv_item.publish(None, {"Address":"o"})

In [ ]:
map = gis.map()

In [ ]:
map.add_layer(csv_lyr)

In [ ]:
map